In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

# Load MovieLens dataset
ratings_data = pd.read_csv('ratings.csv')  # Assuming ratings.csv contains user ratings data
movies_data = pd.read_csv('movies.csv')    # Assuming movies.csv contains movie metadata

# Filter users with more than 10 ratings
user_ratings_count = ratings_data['userId'].value_counts()
active_users = user_ratings_count[user_ratings_count > 10].index
ratings_data_filtered = ratings_data[ratings_data['userId'].isin(active_users)]

# Create user-item matrix with sparse representation
user_item_matrix = ratings_data_filtered.pivot_table(index='userId', columns='movieId', values='rating')
user_item_matrix = user_item_matrix.fillna(0)  # Fill missing values with 0

# Fit Nearest Neighbors clustering
nn_model = NearestNeighbors(n_neighbors=50, metric='cosine')
nn_model.fit(user_item_matrix)

# Recommend movies for a user based on similarity to other users
def recommend_movies(user_id, num_recommendations=5):
    user_ratings = user_item_matrix.loc[user_id].values.reshape(1, -1)
    distances, indices = nn_model.kneighbors(user_ratings)
    similar_users = indices[0][1:]  # Exclude the input user

    # Compute cosine similarity between input user and similar users
    similarities = cosine_similarity(user_ratings, user_item_matrix.iloc[similar_users])

    # Get indices of top similar users based on similarity score
    top_similar_users = np.argsort(similarities[0])[::-1][:num_recommendations]

    # Get movies rated highly by the top similar users
    recommended_movies = user_item_matrix.iloc[similar_users[top_similar_users]].mean(axis=0)
    recommended_movies = recommended_movies.sort_values(ascending=False)[:num_recommendations]

    # Get movie information from movies_data
    recommended_movies_info = movies_data[movies_data['movieId'].isin(recommended_movies.index)]

    return recommended_movies_info

# Example usage:
user_id = 1
recommendations = recommend_movies(user_id)
print(recommendations)


      movieId                                              title  \
257       296                                Pulp Fiction (1994)   
900      1198  Raiders of the Lost Ark (Indiana Jones and the...   
902      1200                                      Aliens (1986)   
1503     2028                         Saving Private Ryan (1998)   
1939     2571                                 Matrix, The (1999)   

                              genres  
257      Comedy|Crime|Drama|Thriller  
900                 Action|Adventure  
902   Action|Adventure|Horror|Sci-Fi  
1503                Action|Drama|War  
1939          Action|Sci-Fi|Thriller  
